# Homework 2
## Introduction and Understanding Pixel Value Scaling, Normalization and Data Augmentation
This homework is focusing on libraries such as PIL, Numpy, and PyTorch.
In the lecture, it is very important to understand that the format of image and each library. Image is in the format of H x W(number of row, number of column), like matrix. Numpy is in the format of (H, W, C), while Pytorch is (C, H, W).
For the image preprocessing part for our neural network, we have to scaling image data to floating point range, and normalize it to floating point range (-1.0, 1.0) to center the data around zero. These steps improve the convergence behavior during training.